### Formatiranje termina u model za prikazivanje

#### Metode za mapiranje

In [1]:
from model_parser import ReadWrite, MeetingSchedule
from model_joined import *
from model_prikaz import *
from parser_utils import *

In [2]:
from collections import defaultdict

In [3]:
import numpy as np
import pandas as pd

In [4]:
def time_grain_index_to_time(
        grain_index: int,
        grains_per_day: int, 
        start_hour: int
) -> str:
    while grain_index > grains_per_day:
        grain_index -= (grains_per_day + 1)
    start_time = start_hour * 60
    time = start_time + grain_index * 15
    hour = time // 60
    minute = time % 60
    return f'{hour:02d}:{minute:02d}'

In [5]:
def minute_to_time(
        minutes: int
) -> str:
    hour = minutes // 60
    minute = minutes % 60
    return f'{hour:02d}:{minute:02d}'

In [6]:
def predavac_to_ispis(
        predavac: Predavac
) -> str:
    if predavac.titula:
        return f'{predavac.prezime} {predavac.titula} {predavac.ime}'
    return f'{predavac.prezime} {predavac.ime}'

In [7]:
def predavac_to_oznaka(
        predavac: Predavac
) -> str:
    return str(predavac.oznaka)

In [8]:
def stud_grupe_to_ispis(
        stud_grupe: list[StudentskaGrupa],
        bi_weekly: bool,
        tip_nastave: str
) -> str:
    # TODO: preuzmi sve grupe za taj studijski program i tu godinu
    # ako su iste duzine, vrati 'SVI', inace vrati spojen ispis grupa
    # bitno da li je svake druge nedelje
    # za sada vraca sve ako je predavanje jer nemamo izborne predmete
    if tip_nastave == 'PRED':
        return 'SVI'
    oznake = [grupa.oznaka for grupa in stud_grupe]
    if bi_weekly:
        first_week = oznake[:len(oznake)//2]
        second_week = oznake[len(oznake)//2:]
        first_week = ', '.join(first_week)
        second_week = ', '.join(second_week)
        return f'{first_week}({second_week})'
    else:
        return ', '.join(oznake)

In [9]:
def meeting_assignment_to_dan(
        ma: MeetingAssignmentJoined
) -> str:
    return ma.startingTimeGrain.dan.danUNedelji

In [10]:
def meeting_assignment_to_prostorija_id(
        ma: MeetingAssignmentJoined
) -> str:
    return ma.prostorija.id

In [11]:
def meeting_assignment_to_prostorija_zauzece(
        ma: MeetingAssignmentJoined
) -> ProstorijaZauzece:
    prostorija_oznaka = ma.prostorija.oznaka
    first_time_grain = ma.startingTimeGrain.grainIndex
    duration_in_grains = ma.meeting.durationInGrains
    last_time_grain = first_time_grain + duration_in_grains
    return ProstorijaZauzece(prostorija_oznaka, first_time_grain, last_time_grain)

In [12]:
def meeting_assignment_to_stud_grupe(
        ma: MeetingAssignmentJoined
) -> list[tuple[str, int]]:
    # lista tuplova => [(studProgramId, godina)]
    stud_program_godine = []
    stud_grupe = ma.meeting.studentskeGrupe
    stud_program_ids = set()
    for grupa in stud_grupe:
        stud_program_id = grupa.studijskiProgram.id
        godina = grupa.godina
        # samo jednom dodajemo informacije o studijskom programu i godini
        if stud_program_id in stud_program_ids:
            continue
        stud_program_ids.add(stud_program_id)
        stud_program_godine.append((stud_program_id, godina))
    return stud_program_godine

In [13]:
def meeting_assignment_to_predavaci_ids(
        ma: MeetingAssignmentJoined
) -> list[str]:
    predavac_id = ma.meeting.predavac.id
    ostali_predavaci = ma.meeting.ostaliPredavaci
    predavaci_ids = [pred.id for pred in ostali_predavaci]
    predavaci_ids.append(predavac_id)
    return predavaci_ids

In [14]:
def meeting_assignment_to_red(
        ma: MeetingAssignmentJoined,
        grains_per_day: int,
        start_hour: int
) -> Red:
    start_time_grain = ma.startingTimeGrain
    # ako optimizator nije uspeo da dodeli vreme
    if start_time_grain is None:
        return None
    start_time_grain = start_time_grain.grainIndex
    duration_in_grains = ma.meeting.durationInGrains
    end_time_grain = start_time_grain + duration_in_grains - 1
    tip_nastave = ma.meeting.meetingTip
    predmet_naziv = ma.meeting.predmet.naziv

    stud_grupa = stud_grupe_to_ispis(ma.meeting.studentskeGrupe, ma.meeting.biWeekly, tip_nastave)
    vreme_pocetka = time_grain_index_to_time(start_time_grain, grains_per_day, start_hour)
    vreme_kraja = time_grain_index_to_time(end_time_grain, grains_per_day, start_hour)
    
    return Red(stud_grupa, vreme_pocetka, vreme_kraja, tip_nastave, predmet_naziv)

In [15]:
def meeting_assignment_to_prostorija_red(
        ma: MeetingAssignmentJoined,
        grains_per_day: int,
        start_hour: int
) -> ProstorijaRed:
    red = meeting_assignment_to_red(ma, grains_per_day, start_hour)
    if red is None:
        return None
    
    stud_grupe = ma.meeting.studentskeGrupe
    # moze za vezbe, za predavanja imamo spojene stud programe - ALI TO NIKOME NIJE BITNO
    odsek = stud_grupe[0].studijskiProgram.naziv
    godina = stud_grupe[0].godina
    semestar_oznaka = stud_grupe[0].semestar
    semestar = arabic_to_roman(godina * 2 - 1 if semestar_oznaka == 'Z' else godina * 2)
    predavac = predavac_to_ispis(ma.meeting.predavac)
    ostali_predavaci = [predavac_to_ispis(predavac) for predavac in ma.meeting.ostaliPredavaci]
    if len(ostali_predavaci):
        predavaci = f'{predavac}, ' + ', '.join(ostali_predavaci)
    else:
        predavaci = predavac
    
    red.odsek = odsek
    red.semestar = semestar
    red.predavac = predavaci
    return ProstorijaRed(**vars(red))

In [16]:
def meeting_assignment_to_predavac_red(
        ma: MeetingAssignmentJoined,
        grains_per_day: int,
        start_hour: int
) -> PredavacRed:
    red = meeting_assignment_to_red(ma, grains_per_day, start_hour)
    if red is None:
        return None

    prostorija = ma.prostorija.oznaka
    stud_grupe = ma.meeting.studentskeGrupe
    # moze za vezbe, za predavanja imamo spojene stud programe - ALI TO NIKOME NIJE BITNO
    odsek = stud_grupe[0].studijskiProgram.naziv
    godina = stud_grupe[0].godina
    semestar_oznaka = stud_grupe[0].semestar
    semestar = arabic_to_roman(godina * 2 - 1 if semestar_oznaka == 'Z' else godina * 2)
    
    red.prostorija = prostorija
    red.odsek = odsek
    red.semestar = semestar
    return PredavacRed(**vars(red))

In [17]:
def meeting_assignment_to_raspored_red(
        ma: MeetingAssignmentJoined,
        grains_per_day: int,
        start_hour: int
) -> RasporedRed:
    red = meeting_assignment_to_red(ma, grains_per_day, start_hour)
    if red is None:
        return None

    prostorija = ma.prostorija.oznaka
    predavac = predavac_to_ispis(ma.meeting.predavac)
    ostali_predavaci = [predavac_to_ispis(predavac) for predavac in ma.meeting.ostaliPredavaci]
    if len(ostali_predavaci):
        predavaci = f'{predavac}, ' + ', '.join(ostali_predavaci)
    else:
        predavaci = predavac
    
    red.prostorija = prostorija
    red.predavac = predavaci
    return RasporedRed(**vars(red))

In [18]:
def meeting_assignment_to_edit_prikaz(
        ma: MeetingAssignmentJoined
) -> EditPrikaz:
    meeting_id = ma.id

    prostorija_kapacitet = ma.prostorija.kapacitet
    potreban_kapacitet = ma.meeting.requiredCapacity
    prekoracen_kapacitet = 0
    if potreban_kapacitet > prostorija_kapacitet:
        prekoracen_kapacitet = potreban_kapacitet - prostorija_kapacitet
    
    nastava_tip = ma.meeting.meetingTip
    
    predmet = ma.meeting.predmet
    predmet_prikaz = PredmetPrikaz(predmet.id, predmet.oznaka, predmet.plan, predmet.naziv)
    
    predavac_fullname = predavac_to_ispis(ma.meeting.predavac)
    predavac_prikaz = PredavacPrikaz(ma.meeting.predavac.id, predavac_fullname)

    stud_grupe = ma.meeting.studentskeGrupe
    stud_programi = set()
    stud_grupe_list = []
    for grupa in stud_grupe:
        stud_program = grupa.studijskiProgram
        stud_programi.add(StudProgram(stud_program.oznaka, stud_program.stepen, stud_program.nivo, grupa.godina))
        stud_grupe_list.append(StudGrupa(grupa.id, grupa.oznaka))
    
    return EditPrikaz(meeting_id, nastava_tip, predmet_prikaz, predavac_prikaz, list(stud_programi), 
                        stud_grupe_list, potreban_kapacitet, prekoracen_kapacitet)

### Kreiranje **Raspored prikaza**

- transformacija ```MeetingAssignment``` u ```StudProgramiRaspored```

In [19]:
def stepen_nivo_to_stud_program_stepen_studija(
        stepen: int, 
        nivo: int
) -> str:
    if stepen == 1 and nivo == 1:
        stepen_studija = "oas"
    if stepen == 1 and nivo == 2:
        stepen_studija = "oss"
    if stepen == 2 and nivo == 1:
        stepen_studija = "mas"
    if stepen == 2 and nivo == 5:
        stepen_studija = "mss"
    return stepen_studija

In [20]:
# u slucaju novih 2-godisnjih master stud programa, dodati ga u listu
# TODO: (izmena da informacija o tome da li je stud program cuva u samom programu)
def is_dvogodisnji_master(
        oznaka: str
) -> bool:
    return oznaka in ["AI1", "AI2", "AI3", "AI4", "AI5", "AI6", "AI7", "AI8"]

In [21]:
def create_dani_dict():
    dani = {}
    for i in range(0, 6):
        dani[i] = RasporedDan(i, [])
    return dani

In [22]:
def create_semestar(
        semestar_num: int,
        start_semestar: int
) -> Semestar:
    semestar = Semestar(start_semestar + (semestar_num - 1) *2, dict())
    semestar.dani = create_dani_dict()
    return semestar

In [23]:
def create_semestri(
        stud_program_raspored: StudProgramRaspored, 
        oznaka: str, 
        stepen: int, 
        nivo: int, 
        start_semestar: int
) -> StudProgramRaspored:
    # oas i oss
    if stepen == 1:
        # 4 semestra za oas, 3 za oss
        for i in range(1, 5):
            semestar = create_semestar(i, start_semestar)
            # semestri[godina] = semestar
            stud_program_raspored.semestri[i] = semestar
            if nivo == 2 and i == 3:
                break
    # mas i mss
    if stepen == 2:
        # 1 semestar za jednogodisnje, 2 za dvogodisnje
        for i in range(1, 3):
            semestar = create_semestar(i, start_semestar)
            # semestri[4+godina] = semestar jer su studentske grupe na masteru 5. i 6. godina
            stud_program_raspored.semestri[4+i] = semestar
            if nivo == 1 and i == 1 and not is_dvogodisnji_master(oznaka):
                break
    return stud_program_raspored

In [24]:
def meeting_assignments_to_stud_programi_raspored(
        semestar_oznaka: str, 
        stud_programi: list[StudijskiProgram], 
        meeting_assignments: list[MeetingAssignmentJoined],
        grains_per_day: int, 
        start_hour: int, 
        unassigned_file_path: str,
        dir_path: str = '../out_data/'
) -> StudProgramiRaspored:
    start_semestar = 1 if semestar_oznaka == "Z" else 2

    # kreiranje strukture
    raspored = StudProgramiRaspored(semestar_oznaka, dict())
    for stud_program in stud_programi:
        stepen_studija = stepen_nivo_to_stud_program_stepen_studija(stud_program.stepen, stud_program.nivo)
        # kreiranje rasporeda za studijski program
        stud_program_raspored = StudProgramRaspored(stud_program.naziv, stepen_studija, dict())
        # dodavanje rasporeda za studijski program u sve rasporede
        raspored.studProgrami[stud_program.id] = stud_program_raspored
        # kreiranje semestara za studijski program
        stud_program_raspored = create_semestri(stud_program_raspored, stud_program.oznaka, stud_program.stepen, stud_program.nivo, start_semestar)
        
    # dodavanje MeetingAssignment-a u dane
    ma_unassigned = []
    for ma in meeting_assignments:
        raspored_red = meeting_assignment_to_raspored_red(ma, grains_per_day, start_hour)
        if raspored_red is None:
            ma_unassigned.append(ma)
            continue
        # neophodno naci u koje sve studijske programe treba dodati red
        # (posto studentske grupe mogu biti iz vise studijskih programa)
        dan_num = meeting_assignment_to_dan(ma)
        stud_program_godine = meeting_assignment_to_stud_grupe(ma)
        # programGodina => (studProgramId, godina)
        for program_godina in stud_program_godine:
            # dodavanje redova
            stud_program_id = program_godina[0]
            godina = program_godina[1]
            stud_program_raspored = raspored.studProgrami[stud_program_id]
            semestar = stud_program_raspored.semestri[godina]
            semestar.dani[dan_num].redovi.append(raspored_red)
    
    # sortiranje po vremenu (9:00-20:00)
    for stud_program_raspored in raspored.studProgrami.values():
        for semestar in stud_program_raspored.semestri.values():
            for dan in semestar.dani.values():
                dan.redovi.sort(key=lambda red: red.vremePocetka)
    
    # zapisivanje nedodeljenih ma za rucnu dodelu
    ReadWrite.write_to_file(ma_unassigned, unassigned_file_path, dir_path)

    return raspored

### Kreiranje **Prostorije raspored prikaza**

- transformacija ```MeetingAssignment``` u ```ProstorijeRaspored```

In [25]:
def meeting_assignments_to_prostorije_raspored(
        semestar: int, 
        prostorije: list[Prostorija], 
        meeting_assignments: list[MeetingAssignmentJoined], 
        grains_per_day: int, 
        start_hour: int, 
        unassigned_file_path: str,
        dir_path: str = '../out_data/'
) -> ProstorijeRaspored:
    # kreiranje strukture
    raspored = ProstorijeRaspored(semestar, dict())
    for prostorija in prostorije:
        prostorija_raspored = ProstorijaRaspored(prostorija.oznaka, create_dani_dict())
        raspored.prostorije[prostorija.id] = prostorija_raspored

    # dodavanje MeetingAssignment-a u dane
    ma_unassigned = []
    for ma in meeting_assignments:
        prostorija_red = meeting_assignment_to_prostorija_red(ma, grains_per_day, start_hour)
        if prostorija_red is None:
            ma_unassigned.append(ma)
            continue
        # dodati red u odgovarajucu prostoriju za odgovarajuci dan
        dan_num = meeting_assignment_to_dan(ma)
        prostorija_id = meeting_assignment_to_prostorija_id(ma)
        prostorija = raspored.prostorije[prostorija_id]
        prostorija.dani[dan_num].redovi.append(prostorija_red)

    # sortiranje po vremenu (9:00-20:00)
    for prosotrija_raspored in raspored.prostorije.values():
        for dan in prosotrija_raspored.dani.values():
            dan.redovi.sort(key=lambda red: red.vremePocetka)
    
    # zapisivanje nedodeljenih ma za rucnu dodelu
    ReadWrite.write_to_file(ma_unassigned, unassigned_file_path, dir_path)

    return raspored

### Kreiranje **Predavaci raspored prikaza**

- transformacija ```MeetingAssignment``` u ```PredavaciRaspored```

In [26]:
def meeting_assignments_to_predavaci_raspored(
        semestar: int, 
        predavaci: list[Predavac], 
        meeting_assignments: list[MeetingAssignmentJoined], 
        grains_per_day: int, 
        start_hour: int, 
        unassigned_file_path: str,
        dir_path: str = '../out_data/'
) -> PredavaciRaspored:
    # kreiranje strukture
    raspored = PredavaciRaspored(semestar, dict())
    for predavac in predavaci:
        predavac_raspored = PredavacRaspored(predavac_to_ispis(predavac), create_dani_dict())
        raspored.predavaci[predavac.id] = predavac_raspored

    # dodavanje MeetingAssignment-a u dane
    ma_unassigned = []
    for ma in meeting_assignments:
        predavac_red = meeting_assignment_to_predavac_red(ma, grains_per_day, start_hour)
        if predavac_red is None:
            ma_unassigned.append(ma)
            continue
        # dodati red svim predavacima za odgovarajuci dan
        dan_num = meeting_assignment_to_dan(ma)
        predavac_ids = meeting_assignment_to_predavaci_ids(ma)
        for predavac_id in predavac_ids:
            predavac = raspored.predavaci[predavac_id]
            predavac.dani[dan_num].redovi.append(predavac_red)

    # sortiranje po vremenu (9:00-20:00)
    for predavacRaspored in raspored.predavaci.values():
        for dan in predavacRaspored.dani.values():
            dan.redovi.sort(key=lambda red: red.vremePocetka)
    
    # zapisivanje nedodeljenih ma za rucnu dodelu
    ReadWrite.write_to_file(ma_unassigned, unassigned_file_path, dir_path)

    return raspored

### Kreiranje **Google kalendar prikaza**

- transformacija ```MeetingAssignment``` u ```GoogleCalendarPrikaz```

In [27]:
from datetime import datetime, timedelta
from copy import copy

# pocetniDan mora biti u formatu '%d/%m/%Y'
def meeting_assignment_to_predavac_google_red(
        ma: MeetingAssignmentJoined,
        grains_per_day: int, 
        start_hour: int, 
        pocetni_dan: str, 
        repeat_num: int = 1):
    datum_format = '%d/%m/%Y'
    # naslov
    meeting = ma.meeting
    tip_nastave = meeting.meetingTip
    predmet_naziv = meeting.predmet.naziv
    naslov = tip_nastave + ', ' + predmet_naziv

    # datumi
    pocetni_dan = datetime.strptime(pocetni_dan, datum_format).date()
    dan_num = meeting_assignment_to_dan(ma)
    datum_pocetka = (pocetni_dan + timedelta(days=dan_num)).strftime(datum_format)
    datum_kraja = datum_pocetka

    # vremena
    start_time_grain = ma.startingTimeGrain.grainIndex
    duration_in_grains = ma.meeting.durationInGrains
    end_time_grain = start_time_grain + duration_in_grains - 1
    vreme_pocetka = time_grain_index_to_time(start_time_grain, grains_per_day, start_hour)
    vreme_kraja = time_grain_index_to_time(end_time_grain, grains_per_day, start_hour)

    # opis
    stud_grupe = meeting.studentskeGrupe
    odsek = stud_grupe[0].studijskiProgram.naziv
    godina = stud_grupe[0].godina
    stud_grupa = stud_grupe_to_ispis(stud_grupe, meeting.biWeekly, tip_nastave)
    opis = odsek + ', ' + str(godina) + '. godina, grupe: ' + stud_grupa

    # lokacija
    lokacija = ma.prostorija.oznaka

    cal_prikaz = GoogleCalendarPrikaz(naslov, datum_pocetka, datum_kraja, vreme_pocetka, vreme_kraja, opis, lokacija, False, False)
    # vrati listu termina * broj nedelja
    # uvecaj datum za 7 dana za svaku nedelju
    prikazi = [cal_prikaz]
    for i in range(repeat_num - 1):
        cal_next_prikaz = copy(cal_prikaz)
        # (0 + 1) * 7 = pomera_nedelja_unapred
        # pomeraj_nedelja_unapred + pomeraj_od_pocetnog_dana
        sledeciDan = (pocetni_dan + timedelta(days=(i + 1) * 7 + dan_num)).strftime(datum_format)
        cal_next_prikaz.datumPocetka = sledeciDan
        cal_next_prikaz.datumKraja = sledeciDan
        prikazi.append(cal_next_prikaz)
    
    return prikazi

In [28]:
def meeting_assignments_to_predavac_google_calendar(
        meeting_assignments: list[MeetingAssignmentJoined],
        predavac_id: str,
        pocetni_dan: str, 
        repeat_num: int,
        grains_per_day: int, 
        start_hour: int
):
    raspored = []
    for ma in meeting_assignments:
        if ma.startingTimeGrain is None:
            continue
        predavac_ids = meeting_assignment_to_predavaci_ids(ma)
        if predavac_id in predavac_ids:
            raspored.extend(meeting_assignment_to_predavac_google_red(ma, grains_per_day, start_hour, pocetni_dan, repeat_num))
    return raspored

In [29]:
def write_google_calendar_csv(
        filePath: str,
        meeting_assignments: list[MeetingAssignmentJoined], 
        predavac_id: str, 
        pocetni_dan: str, 
        repeat_num: int, 
        grains_per_day: int, 
        start_hour: int
) -> None:
    # columns = ['Subject', 'Start Date', 'Start Time', 'End Date', 'End Time', 'All Day Event', 'Description', 'Location', 'Private']
    raspored = meeting_assignments_to_predavac_google_calendar(
        meeting_assignments, predavac_id, pocetni_dan, repeat_num, grains_per_day, start_hour)
    df = pd.DataFrame.from_records([r.to_dict() for r in raspored])
    df.to_csv(filePath, index=False)

### Kreiranje prikaza za **Edit-ovanje**

- zauzece prostorija po time-grains (podeoci od 15 minuta)
- prikaz celog mitinga u svakom time-grain koji zauzima

In [30]:
def generate_grains_per_day(
        start_idx: int, 
        count_per_day: int
) -> list[int]:
    day = []
    for j in range(count_per_day):
        day.append(start_idx * count_per_day + j)
    return day

In [31]:
def generate_grains(
        day_count: int, 
        count_per_day: int
) -> tuple[list[int], list[int]]:
    grains_per_day = []
    all_grains = []
    for i in range(day_count):
        day = []
        day = generate_grains_per_day(i, count_per_day)
        grains_per_day.append(day)
        all_grains.extend(day)
    return grains_per_day, all_grains

In [42]:
def meeting_assignments_to_edit_view(
        meeting_assignments: list[MeetingAssignmentJoined], 
        day_num: int, 
        grains_per_day: int
) -> dict[str, dict[int, EditPrikaz]]: # TODO: mozda i ne, proveriti da li ce to biti kreirano
    grains_per_day, all_grains = generate_grains(day_num, grains_per_day)
    prostorija_odrzavanje = defaultdict(lambda: all_grains.copy())
    for ma in meeting_assignments:
        if ma.startingTimeGrain is None:
            continue
        zauzece = meeting_assignment_to_prostorija_zauzece(ma)
        # dodajemo zauzece na odgovarajuci time grain
        # -> vreme kada su slobodne ostaju indeksi
        odrzavanje = meeting_assignment_to_edit_prikaz(ma)
        for timeGrain in range(zauzece.firstTimeGrainIndex, zauzece.lastTimeGrainIndex):
            prostorija_odrzavanje[zauzece.prostorijaOznaka][timeGrain] = odrzavanje
    return prostorija_odrzavanje


In [33]:
def create_columns(
        grains_per_day: int, 
        start_hour: int
) -> list[str]:
    grains = generate_grains_per_day(0, grains_per_day)
    # dodavanje kolone za oznaku
    columns = ['Oznaka'] + [time_grain_index_to_time(grain, grains_per_day, start_hour) for grain in grains]
    return columns

In [34]:
def create_prostorija_row(
        day_num: int, 
        grains_per_day: int, 
        weekly_zauzece # TODO: sta si ti?
): # TODO: sta ti vracas?
    # dobavi sve za odredjeni dan
    start_grain = day_num * grains_per_day
    end_grain = start_grain + grains_per_day
    daily_zauzece = weekly_zauzece[start_grain:end_grain]
    # napravi lep to string za objekat
    daily_prikaz = [str(zauzece) if type(zauzece) != int else '' for zauzece in daily_zauzece]
    return daily_prikaz

In [35]:
def create_all_rows(
        day_num: int, 
        grains_per_day: int, 
        all_zauzeca # TODO: sta ste vi
): # TODO: sta vracas
    all_rows = []
    for oznaka, prostorija_zauzece in all_zauzeca.items():
        all_rows.append([str(oznaka)] + create_prostorija_row(day_num, grains_per_day, prostorija_zauzece))
    return all_rows

In [36]:
def create_day_edit(
        day_num: int, 
        grains_per_day: int, 
        start_hour: int, 
        all_zauzeca # TODO: nesto
): # TODO: vraca nesto
    columns = create_columns(grains_per_day, start_hour)
    rows = create_all_rows(day_num, grains_per_day, all_zauzeca)
    return columns, rows

In [37]:
def style_cell(
        value: str, 
        ok, warn, danger # TODO: otkriti kog su tipa formati
):
    if 'PRED' in value and 'prekoracenje: 0' not in value:
        return warn
    if ('AUD' in value or 'RAC' in value or 'LAB' in value) and 'prekoracenje: 0' not in value:
        return danger
    if 'prekoracenje: 0' in value:
        return ok

In [38]:
def write_edit_view(
        prostorija_odrzavanje, # nesto
        file_name: str,
        dir_path: str = '../out_data/'
):
    grains_per_day = 60 + 1 # 60 grains po danu + 1 za poslednju pauzu
    start_hour = 7
    dani = ['PON', 'UTO', 'SRE', 'CET', 'PET', 'SUB']

    with pd.ExcelWriter(dir_path + file_name + '.xlsx', engine='xlsxwriter') as writer:
        for idx, dan in enumerate(dani):
            columns, rows = create_day_edit(idx, grains_per_day, start_hour, prostorija_odrzavanje)
            df = pd.DataFrame(rows, columns=columns)
            df.to_excel(writer, sheet_name=dan, index=False)
            # formatiranje
            workbook = writer.book
            # column width
            worksheet = writer.sheets[dan]
            worksheet.set_default_row(188.5)
            worksheet.set_row(0, 14.5)
            worksheet.set_column('B:BJ', 20)
            # freeze header and first column
            worksheet.freeze_panes(1, 1)
            # merge cells
            merge_ok_format = workbook.add_format({'align': 'center', 'valign': 'vcenter', 'bg_color': '#D8E4BC', 'border': 1, 'text_wrap': True})
            merge_warn_format = workbook.add_format({'align': 'center', 'valign': 'vcenter', 'bg_color': '#FDE9D9', 'border': 1, 'text_wrap': True})
            merge_danger_format = workbook.add_format({'align': 'center', 'valign': 'vcenter', 'bg_color': '#E6B8B7', 'border': 1, 'text_wrap': True})

            for row_idx, row in enumerate(rows):
                # za svaki red
                current_meeting = ''
                merge_start = 0
                merge_end = 0
                for col_idx, meeting in enumerate(row):
                    # ignorisi prvu vrednost (ime prostorije)
                    if col_idx == 0:
                        continue
                    # zapamti prvo pojavljivanje (ime i broj kolone -> B2)
                    if current_meeting == '' and meeting != '':
                        current_meeting = meeting
                        merge_start = col_idx
                        merge_end = merge_start
                    
                    # sve dok je isto, uvecavaj broj kolone (B2++)
                    if current_meeting == meeting:
                        merge_end = col_idx
                    # naisao na novo
                    else:
                        # odradi merge
                        style = style_cell(current_meeting, merge_ok_format, merge_warn_format, merge_danger_format)
                        worksheet.merge_range(row_idx+1, merge_start, row_idx+1, merge_end, current_meeting, style)
                        # resetuj vrednosti na novi
                        current_meeting = meeting
                        merge_start = col_idx
                        merge_end = merge_start
                # merge poslednjeg ako nije prazno
                if current_meeting != '':
                    style = style_cell(current_meeting, merge_ok_format, merge_warn_format, merge_danger_format)
                    worksheet.merge_range(row_idx+1, merge_start, row_idx+1, merge_end, current_meeting, style)

### Izvršavanje

In [50]:
schedule = MeetingSchedule.read_entity_from_file('7_schedule_oas')
meeting_assignments = MeetingAssignmentJoined.read_list_from_file('8_optimized_oas')

In [51]:
unassigned_file_path = '9_unassigned_schedule_oas'
grains_per_day = 60
start_hour = 7
# neki predavac do kog je stigla optimizacija
predavac_id = next(predavac.id for predavac in schedule.predavacList if predavac.ime == 'Petar' and predavac.prezime == 'Trifunović')
pocetni_dan = '16/10/2023'
broj_nedelja = 2

In [52]:
raspored = meeting_assignments_to_stud_programi_raspored('Z', schedule.studProgramList, meeting_assignments, grains_per_day, start_hour, unassigned_file_path)
prostorije_raspored = meeting_assignments_to_prostorije_raspored('Z', schedule.prostorijaList, meeting_assignments, grains_per_day, start_hour, unassigned_file_path)
predavaci_raspored = meeting_assignments_to_predavaci_raspored('Z', schedule.predavacList, meeting_assignments, grains_per_day, start_hour, unassigned_file_path)

predavac_kalendar = meeting_assignments_to_predavac_google_calendar(meeting_assignments, predavac_id, pocetni_dan, broj_nedelja, grains_per_day, start_hour)

# # TODO: proveriti sto je ovde veci broj
day_num = 6
grains_per_day = 61
prostorija_odrzavanje = meeting_assignments_to_edit_view(meeting_assignments, day_num, grains_per_day)

In [53]:
ReadWrite.write_to_file(raspored, '9_raspored')
ReadWrite.write_to_file(prostorije_raspored, '9_prostorije_raspored')
ReadWrite.write_to_file(predavaci_raspored, '9_predavaci_raspored')

file_name = '9_editRasporedView'
write_edit_view(prostorija_odrzavanje, file_name)

In [54]:
siit_id = next(x.id for x in schedule.studProgramList if x.naziv == 'Softversko inženjerstvo i informacione tehnologije')
siit = raspored.studProgrami[siit_id]
print(siit)

predavac = predavaci_raspored.predavaci[predavac_id]
print(predavac)

print(predavac_kalendar)

StudProgramRaspored(studProgramNaziv='Softversko inženjerstvo i informacione tehnologije', studProgramStepenStudija='oas', semestri={1: Semestar(semestarNum=1, dani={0: RasporedDan(danNum=0, redovi=[RasporedRed(studGrupa='3', vremePocetka='10:15', vremeKraja='12:45', vrstaNastave='RAC', nazivPred='Arhitektura računara', prostorija='MI A2-3', predavac='Trifunović Petar'), RasporedRed(studGrupa='3', vremePocetka='13:30', vremeKraja='16:00', vrstaNastave='RAC', nazivPred='Osnove programiranja', prostorija='MI B4-5A', predavac='Nikolić Lazar')]), 1: RasporedDan(danNum=1, redovi=[RasporedRed(studGrupa='SVI', vremePocetka='09:30', vremeKraja='11:15', vrstaNastave='PRED', nazivPred='Sociologija tehnike', prostorija='310', predavac='Nešić Tomašević dr Ana'), RasporedRed(studGrupa='2', vremePocetka='21:00', vremeKraja='08:15', vrstaNastave='RAC', nazivPred='Osnove programiranja', prostorija='NTP-309', predavac='Nikolić Lazar')]), 2: RasporedDan(danNum=2, redovi=[RasporedRed(studGrupa='2', vreme